 # Backtesting Trading Strategies

 ## Table of Contents
 1. [Import Libraries](#1)
 2. [Backtesting Functions](#2)
 3. [Load Models and Data](#3)
 4. [Perform Backtest Across All Models and Pairs](#4)
 5. [Analyze Results](#5)
 6. [Backtest Transformer Model Results](#6)

 ## 1. Import Libraries <a id='1'></a>

In [36]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import logging
import pickle
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
import yfinance as yf
import dataframe_image as dfi

plt.style.use("custom_onedark")

 ## 2. Backtesting Functions <a id='2'></a>

In [24]:
def calculate_signals(df, threshold):
    df['long_spread'] = df['z'] < -threshold
    df['short_spread'] = df['z'] > threshold
    return df

def update_hedge_ratio(model, s1, s2, offset=0):
    X = s1.reshape(-1, 1)
    y = s2 + offset
    model.fit(X, y)
    return model.coef_[0]

def run_backtest(df, initial_capital, model, ratio_offset=0, transaction_cost=0):
    df['position_S1'] = 0
    df['position_S2'] = 0
    df['holdings_S1'] = 0
    df['holdings_S2'] = 0
    df['cash'] = initial_capital
    df['pnl'] = 0
    df['transaction_costs'] = 0

    for i in range(1, len(df)):
        hedge_ratio = update_hedge_ratio(model, df['S1'].values[:i], df['S2'].values[:i], offset=ratio_offset)
        df.at[i, 'hedge_ratio'] = hedge_ratio

        # Calculate new positions
        new_position_S1 = 0
        new_position_S2 = 0
        if df.iloc[i]['long_spread']:
            new_position_S1 = 1
            new_position_S2 = -hedge_ratio
        elif df.iloc[i]['short_spread']:
            new_position_S1 = -1
            new_position_S2 = hedge_ratio

        # Calculate transaction costs
        transaction_cost_S1 = abs(new_position_S1 - df.iloc[i - 1]['position_S1']) * df.iloc[i]['S1'] * transaction_cost
        transaction_cost_S2 = abs(new_position_S2 - df.iloc[i - 1]['position_S2']) * df.iloc[i]['S2'] * transaction_cost
        df.at[i, 'transaction_costs'] = transaction_cost_S1 + transaction_cost_S2

        # Update positions
        df.at[i, 'position_S1'] = new_position_S1
        df.at[i, 'position_S2'] = new_position_S2
        df.at[i, 'holdings_S1'] = df.iloc[i]['position_S1'] * df.iloc[i]['S1']
        df.at[i, 'holdings_S2'] = df.iloc[i]['position_S2'] * df.iloc[i]['S2']
        df.at[i, 'cash'] = df.iloc[i - 1]['cash'] - df.iloc[i]['holdings_S1'] - df.iloc[i]['holdings_S2'] - df.iloc[i]['transaction_costs']
        df.at[i, 'pnl'] = df.iloc[i]['cash'] + df.iloc[i]['holdings_S1'] + df.iloc[i]['holdings_S2']

    return df.iloc[1:]


def calculate_cagr(portfolio, initial_capital):
    final_portfolio = portfolio['cash'].iloc[-1]
    delta_days = (portfolio.index[-1] - portfolio.index[0]).days
    YEAR_DAYS = 365
    cagr = (final_portfolio / initial_capital) ** (YEAR_DAYS / delta_days) - 1
    return cagr

def calculate_backtest_metrics(backtest, initial_capital, verbose=False):
    final_portfolio = backtest['pnl'].iloc[-1]
    delta_days = (backtest.index[-1] - backtest.index[0]).days
    YEAR_DAYS = 365

    returns = backtest['pnl'].pct_change()
    downside_returns = returns[returns < 0]
    cumulative_returns = (1 + returns).cumprod()

    metrics = {
        'Number of days': delta_days,
        'Final Portfolio Value': f'{final_portfolio:.2f}',
        'Annualized Returns': (final_portfolio / initial_capital) ** (YEAR_DAYS / delta_days) - 1,
        'Sharpe Ratio': (returns.mean() / returns.std()) * (YEAR_DAYS ** 0.5),
        'Sortino Ratio': (returns.mean() / downside_returns.std()) * (YEAR_DAYS ** 0.5),
        'Max Drawdown': (cumulative_returns.div(cumulative_returns.cummax()) - 1).min(),
        'Average Return': returns.mean(),
        'Standard Deviation': returns.std(),
        'CAGR': calculate_cagr(backtest, initial_capital)}

    if verbose:
        output = ""
        for key, value in metrics.items():
            if isinstance(value, float):
                value = round(value, 2)
            width = len(str(value)) + 4
            output += "{:5s}: {:>{}} | ".format(key, str(value), width)
        print(output)

    return metrics

 ## 3. Load Models and Data <a id='3'></a>

In [25]:
all_models_path = './model_predictions_all_pairs.pkl'
with open(all_models_path, 'rb') as f:
    lstm_comp_result_per_ticker = pickle.load(f)

 ## 4. Perform Backtest Across All Models and Pairs <a id='4'></a>

In [26]:
def process_model_data(lstm_comp_result, model_idx):
    try:
        X_train, y_train, X_test, y_test = lstm_comp_result[model_idx]['data']
        if isinstance(X_train, np.ndarray) and X_train.ndim == 2:
            X_test = pd.DataFrame(X_test)
    except Exception as e:
        logging.error(f"Error occurred while retrieving data for model {model_idx}: {str(e)}")
    return X_train, y_train, X_test, y_test

def extract_X_test_data(lstm_comp_result, model_idx, X_test):
    if not isinstance(X_test, pd.DataFrame):
        x_test_key_identifiers = ['X_test_bkp', 'X_test_og']
        backup_key = next((key for key in x_test_key_identifiers if key in lstm_comp_result[model_idx]), None)
        X_test = pd.DataFrame(lstm_comp_result[model_idx][backup_key])
    return X_test.copy()[['Adj Close_S1', 'Adj Close_S2']].set_axis(['S1', 'S2'], axis=1)

def process_backtest(lstm_comp_result, model_idx, y_pred, X_test, threshold, initial_capital, transaction_costs=0):
    data_backtest = extract_X_test_data(lstm_comp_result, model_idx, X_test)
    data_backtest['z'] = y_pred
    data_backtest = data_backtest.reset_index()

    model = BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06)
    signals = calculate_signals(data_backtest.copy(), threshold)
    portfolio = run_backtest(signals.copy(), initial_capital, model, transaction_cost)
    portfolio = portfolio.set_index('Date')
    metrics = calculate_backtest_metrics(portfolio, initial_capital, verbose=False)
    cagr = calculate_cagr(portfolio, initial_capital)

    return {
        'portfolio': portfolio,
        'metrics': metrics,
        'cagr': cagr
    }


def process_lstm_results(lstm_comp_result_per_ticker, threshold,
                         initial_capital, transaction_costs=0):
    ticker_key_ids = list(lstm_comp_result_per_ticker.keys())
    backtest_portfolio_per_result = {}
    portfolio_per_pair = {}

    for ticker_key_idx in ticker_key_ids:
        lstm_comp_result = lstm_comp_result_per_ticker[ticker_key_idx]
        try:
            lstm_comp_result['LSTM']['X_test_og'] = lstm_comp_result[
                'Vanilla LSTM']['X_test_bkp']
            lstm_comp_result['BiLSTM with Dropout'][
                'X_test_og'] = lstm_comp_result['Vanilla LSTM']['X_test_bkp']
        except:
            pass
        for model_idx in lstm_comp_result.keys():
            X_train, y_train, X_test, y_test = process_model_data(lstm_comp_result, model_idx)
            y_pred = lstm_comp_result[model_idx]['y_pred']
            backtest_portfolio_per_result[model_idx] = process_backtest(
                lstm_comp_result, model_idx, y_pred, X_test, threshold,
                initial_capital)

        metric_tbl = {}
        for model_idx, model_res in backtest_portfolio_per_result.items():

            metric_tbl[model_idx] = model_res['metrics']
            metric_tbl[model_idx].update(
                lstm_comp_result[model_idx]['metrics'])

        portfolio_per_pair[ticker_key_idx] = metric_tbl

    backtest_batch_results = pd.DataFrame(metric_tbl).T
    return backtest_batch_results, portfolio_per_pair


threshold = 0.4
initial_capital = 100_000
transaction_cost = 0.001
backtest_batch_results, portfolio_per_pair = process_lstm_results(lstm_comp_result_per_ticker, threshold, initial_capital, transaction_cost)
backtest_batch_results

,Number of days,Final Portfolio Value,Annualized Returns,Sharpe Ratio,Sortino Ratio,Max Drawdown,Average Return,Standard Deviation,CAGR,r2,mae,mse,rmse
LinearRegression,587,358425.53,1.211711,6.858112,16.873235,-0.222065,0.003212,0.008948,1.212203,-2067341297368747737088.0,45635496238.328552,3147210444795183366144.0,56100003964.306305
GradientBoostingRegressor,587,180549.02,0.443951,2.662006,3.835669,-0.349481,0.001527,0.010958,0.443313,0.658238,0.572943,0.52028,0.721304
RandomForestRegressor,587,150277.44,0.288231,1.862652,2.579316,-0.402639,0.001071,0.010987,0.287547,0.688469,0.547859,0.474258,0.688664
LGBMRegressor,587,153462.12,0.305139,1.869182,2.668937,-0.400135,0.00113,0.011545,0.304461,0.663403,0.554689,0.512418,0.715834
Vanilla LSTM,489,118982.87,0.138527,0.660995,0.462073,-0.811647,0.000767,0.022155,0.138527,-0.124709,1.147142,1.787201,1.336862
LSTM,489,221007.24,0.807475,4.131753,7.447116,-0.20965,0.002423,0.011202,0.807231,0.456536,0.817688,0.863582,0.929291
BiLSTM with Dropout,489,203613.07,0.700196,3.342927,4.338012,-0.241471,0.002194,0.012538,0.699947,0.408672,0.835791,0.93964,0.96935


 ## 5. Analyze Results <a id='5'></a>

In [27]:
dfs = []
for key, tbl in portfolio_per_pair.items():
    df1 = pd.DataFrame(tbl).T
    dfs.append(df1['Annualized Returns'].to_frame(name=f'{key[0]}_{key[1]}').T)
res = pd.concat(dfs)

agg_YoY_returns = res.applymap(lambda x: f"{x*100:.2f}%").replace('nan%', '').rename_axis('Agg_YoY_Returns', axis=1)
display(agg_YoY_returns)

Agg_YoY_Returns,LinearRegression,GradientBoostingRegressor,RandomForestRegressor,LGBMRegressor,Vanilla LSTM,LSTM,BiLSTM with Dropout
GS_BLK,-20.18%,35.29%,5.36%,-59.80%,0.94%,22.70%,43.11%
JPM_CRM,-24.81%,-19.76%,-7.59%,-5.65%,-0.29%,5.19%,-3.71%
HON_GS,,3.38%,3.51%,1.25%,30.11%,12.32%,25.99%
HON_BLK,400.29%,,,,89.72%,45.21%,67.55%
JPM_GS,,14.42%,33.96%,9.91%,5.52%,-19.03%,26.17%
WFC_AXP,-6.77%,-10.55%,-6.74%,3.71%,25.40%,27.63%,37.75%
WFC_GS,382.04%,384.03%,380.47%,380.10%,83.16%,35.32%,127.93%
WFC_MS,-11.88%,1.99%,-1.22%,3.87%,1.04%,0.75%,4.86%
JPM_GOOGL,22.09%,6.60%,4.29%,0.58%,10.03%,4.14%,7.00%
INTC_BLK,38.14%,4.08%,5.01%,0.22%,656.38%,814.46%,422.48%


In [28]:
backtest_batch_results = backtest_batch_results

backtest_batch_results.iloc[0] = pd.to_numeric(
    backtest_batch_results.iloc[0], errors='coerce'
).map('{:.2e}'.format)

backtest_batch_results = backtest_batch_results.round(2)
backtest_batch_results['Final Portfolio Value'] = pd.to_numeric(
    backtest_batch_results['Final Portfolio Value'], errors='coerce'
).map('${:,.2f}'.format)

for col in ['CAGR', 'Annualized Returns', 'Average Return']:
    backtest_batch_results[col] = pd.to_numeric(
        backtest_batch_results[col], errors='coerce'
    ).map('{:.2%}'.format)

pd.options.display.float_format = '{:.3f}'.format
df = backtest_batch_results.round(3)

backtest_batch_results.to_csv('aggregate_backtest_results_across_pairs.csv')
dfi.export(df[:], 'portfolio_aggregated_pairs.png')

| Model                     | Number of days | Final Portfolio Value | Annualized Returns | Sharpe Ratio | Sortino Ratio | Max Drawdown | Average Return | Standard Deviation | CAGR        | r2              | mae              | mse              | rmse             |
|---------------------------|----------------|----------------------:|-------------------:|-------------:|--------------:|-------------:|---------------:|-------------------:|------------:|----------------:|----------------:|----------------:|----------------:|
| GradientBoostingRegressor | 587            |            $180,549.02 |             44.40% |       2.6620 |        3.8357 |      -34.95% |         0.15% |             1.10% |     44.33% |          0.6582 |          0.5729 |          0.5203 |          0.7213 |
| RandomForestRegressor     | 587            |            $150,277.44 |             28.82% |       1.8627 |        2.5793 |      -40.26% |         0.11% |             1.10% |     28.75% |          0.6885 |          0.5479 |          0.4743 |          0.6887 |
| LGBMRegressor             | 587            |            $153,462.12 |             30.51% |       1.8692 |        2.6689 |      -40.01% |         0.11% |             1.15% |     30.45% |          0.6634 |          0.5547 |          0.5124 |          0.7158 |
| Vanilla LSTM              | 489            |            $118,982.87 |             13.85% |       0.6610 |        0.4621 |      -81.16% |         0.08% |             2.22% |     13.85% |         -0.1247 |          1.1471 |          1.7872 |          1.3369 |
| LSTM                      | 489            |            $221,007.24 |             80.75% |       4.1318 |        7.4471 |      -20.96% |         0.24% |             1.12% |     80.72% |          0.4565 |          0.8177 |          0.8636 |          0.9293 |
| BiLSTM with Dropout       | 489            |            $203,613.07 |             70.02% |       3.3429 |        4.3380 |      -24.15% |         0.22% |             1.25% |     69.99% |          0.4087 |          0.8358 |          0.9396 |          0.9694 |

### Compare performance to SPY

In [29]:
## get CAGR of SPX over the same period
import yfinance as yf
spx = yf.Ticker('^GSPC')
spx_hist = spx.history(start='2021-01-01', end='2021-12-31')
spx_hist['Close'].pct_change().cumsum()


Date
2021-01-04 00:00:00-05:00     NaN
2021-01-05 00:00:00-05:00   0.007
2021-01-06 00:00:00-05:00   0.013
2021-01-07 00:00:00-05:00   0.028
2021-01-08 00:00:00-05:00   0.033
                             ... 
2021-12-23 00:00:00-05:00   0.253
2021-12-27 00:00:00-05:00   0.267
2021-12-28 00:00:00-05:00   0.266
2021-12-29 00:00:00-05:00   0.267
2021-12-30 00:00:00-05:00   0.264
Name: Close, Length: 251, dtype: float64

In [ ]:
model_idx = list(backtest_batch_results.keys())[0]
portfolio = portfolio_per_pair[model_idx]['portfolio']
start_date, end_date = portfolio.index[0], portfolio.index[-1]

spx_hist = spx.history(start=start_date, end=end_date)

# Calculate CAGR and annualized returns of SPX
initial_capital = 100_000
spx_portfolio = initial_capital * (spx_hist['Close'] / spx_hist['Close'][0])

# Calculate CAGR
spx_cagr = (spx_portfolio[-1] / spx_portfolio[0]) ** (1 / ((spx_portfolio.index[-1] - spx_portfolio.index[0]).days / 365)) - 1

# Calculate annualized returns
spx_returns = spx_portfolio.pct_change().dropna()
spx_annualized_returns = ((1 + spx_returns).prod()) ** (252 / len(spx_returns)) - 1

print(f"SPX CAGR: {spx_cagr:.4f}")
print(f"SPX Annualized Returns: {spx_annualized_returns:.4f}")


 ## 6. Backtest Transformer Model Results <a id='6'></a>

In [ ]:
import os
os.chdir('/Users/darien/Library/Mobile Documents/com~apple~CloudDocs/Code/QuantTrading/TradingProject/milestone-3/Trading-Strategy-Project/strategies/backtesting/scratch/')
with open('../../results/resultsby_pair.pkl', 'rb') as f:
    results_transformer = pickle.load(f)

with open('./X_testby_pair.pickle', 'rb') as f:
    X_test_transformer = pickle.load(f)

charles = pd.read_csv('./X_test_charles.csv')
charles = charles[['Date','Adj Close_S1', 'Adj Close_S2']]

backtest_results_transformer = {}

for i in range(len(results_transformer)):
    try:
        trans_result = results_transformer[i]
        y_test = trans_result.get('Y_test')
        tmp_df = charles.copy()

        X_test = X_test_transformer[i]
        y_test = results_transformer[i].get('Y_test')

        tmp = pd.DataFrame()
        tmp.index = X_test.index
        tmp['S1'] = X_test['Adj Close_S1']
        tmp['S2'] = X_test['Adj Close_S2']

        tmp = tmp.iloc[-len(y_test):]
        tmp['z'] = y_test
        tmp = tmp.reset_index()

        threshold = 1
        initial_capital = 100000
        model = BayesianRidge()

        signals = calculate_signals(tmp.copy(), threshold)
        portfolio = run_backtest(signals.copy(), initial_capital, model)
        portfolio = portfolio.set_index('Date')

        metrics = calculate_backtest_metrics(portfolio, initial_capital, verbose=True)
        cagr = calculate_cagr(portfolio, initial_capital)

        backtest_results_transformer[f'pair_{i}'] = {'metrics': metrics, 'cagr': cagr, 'portfolio': portfolio}
    except:
        print(f"Error in pair {i}")

Number of days:     398 | Final Portfolio Value:     124652.49 | Annualized Returns:     0.22 | Sharpe Ratio:     1.12 | Sortino Ratio:     5.58 | Max Drawdown:     -0.13 | Average Return:     0.0 | Standard Deviation:     0.02 | CAGR :     0.24 | 
Number of days:     300 | Final Portfolio Value:     102271.53 | Annualized Returns:     0.03 | Sharpe Ratio:     1.38 | Sortino Ratio:     1.47 | Max Drawdown:     -0.03 | Average Return:     0.0 | Standard Deviation:     0.0 | CAGR :     0.03 | 
Error in pair 2
Error in pair 3
Number of days:     320 | Final Portfolio Value:     79451.78 | Annualized Returns:     -0.23 | Sharpe Ratio:     -9.18 | Sortino Ratio:     -8.77 | Max Drawdown:     -0.21 | Average Return:     -0.0 | Standard Deviation:     0.0 | CAGR :     -0.24 | 
Number of days:     320 | Final Portfolio Value:     -64596.68 | Annualized Returns:     nan | Sharpe Ratio:     -0.93 | Sortino Ratio:     -0.54 | Max Drawdown:     -1.58 | Average Return:     -0.01 | Standard Deviatio

In [ ]:
backtest_average_transformer_metrics = {metric: 0 for metric in backtest_results_transformer.get('pair_0').get('metrics').keys()}

for pair in backtest_results_transformer.keys():
    if pair in ['pair_4', 'pair_19','pair_7','pair_4', 'pair_18']:
        continue
    for metric in backtest_results_transformer.get(pair).get('metrics').keys():
        metric_value = backtest_results_transformer.get(pair).get('metrics').get(metric)
        if isinstance(metric_value, (int, float)) and metric_value > 0 or metric == 'Number of days':
            backtest_average_transformer_metrics[metric] += metric_value
        else:
            backtest_average_transformer_metrics[metric] += 0

# Round the values
agg_transformer_backtest_results = pd.DataFrame([backtest_average_transformer_metrics]).round(4)
print(agg_transformer_backtest_results.T.to_markdown())

|                       |         0 |
|:----------------------|----------:|
| Number of days        | 3088      |
| Final Portfolio Value |    0      |
| Annualized Returns    |    0.5564 |
| Sharpe Ratio          |   18.5693 |
| Sortino Ratio         |  246.507  |
| Max Drawdown          |    0      |
| Average Return        |    3.0226 |
| Standard Deviation    |   50.2229 |
| CAGR                  |    0.5741 |
